In [35]:
import os
import pandas as pd
from BaselineRemoval import BaselineRemoval
from scipy import signal
import similarity as sim

def find_peaks(y):
    dist, prom = 500, 500
    window, deg = 15, 2     
    smoothed = signal.savgol_filter(y, window, deg, deriv=0)
    brm = BaselineRemoval(smoothed)
    z = brm.ZhangFit(lambda_=400,repitition=15, porder=1)
    peaks,_ = signal.find_peaks(z, distance=dist, prominence=prom)
    return peaks, z

DATA_DIR_PATH = "datasets/export"
# file1 = "L187_0_B1_1.txt"
file1 = "L138_0_G4_1.txt"
# file2 = "L185_0_A10_1.txt"
file2 = "L291_0_A1_1.txt"

df1 = pd.read_table(f"{DATA_DIR_PATH}/{file1}",sep=" ", header=None,names=['m/z', 'intensity']) 
df2 = pd.read_table(f"{DATA_DIR_PATH}/{file2}",sep=" ", header=None,names=['m/z', 'intensity']) 

x = df1['m/z'].to_numpy()
y = df1['intensity'].to_numpy()
adist, prom = 500, 500
peaks, z =  find_peaks(y)
print(f'{file1} peaks: {len(peaks)}')
pk1 = (x[peaks], z[peaks])

x = df2['m/z'].to_numpy()
y = df2['intensity'].to_numpy()
peaks, z =  find_peaks(y)
pk2 = (x[peaks], z[peaks])
print(f'{file2} peaks: {len(peaks)}')

L138_0_G4_1.txt peaks: 65
L291_0_A1_1.txt peaks: 47


In [37]:
dic1 = sim._peak(pk1,d=1)
dic2 = sim._peak(pk2,d=1)
rnk1 = sim._rank(dic1)
rnk2 = sim._rank(dic2)
rs = dict()
for k in rnk1.keys() & rnk2.keys():
    rs[k] = (rnk1[k], rnk2[k])
for k, v in sorted(rs.items(), key=lambda x: x[1][0]):
    print (k,v)
for k in rnk1.keys() - rnk2.keys():
    print(f'{k}\t {rnk1[k]}, N/A')
for k in rnk2.keys() - rnk1.keys():
    print(f'{k}\t  N/A, {rnk2[k]}')

4870 (1, 2)
9750 (2, 1)
4320 (3, 3)
6360 (4, 4)
7400 (5, 5)
3700 (6, 8)
4520 (7, 7)
2750 (9, 6)
6010 (10, 10)
7020 (11, 19)
2160 (13, 12)
9040 (14, 14)
3510 (15, 26)
6860 (16, 20)
3000 (17, 33)
4690 (18, 24)
8090 (19, 37)
1580 (20, 16)
1170 (22, 15)
1850 (23, 9)
9390 (24, 35)
1650 (25, 21)
10340 (26, 39)
1330 (27, 31)
1450 (28, 18)
2260 (29, 28)
7940 (34, 17)
1520 (36, 25)
1940 (38, 27)
2460 (40, 41)
2080 (42, 34)
6220 (44, 32)
1780 (46, 36)
2560 (49, 46)
2650 (50, 43)
5120	 12, N/A
5380	 41, N/A
10500	 65, N/A
13580	 52, N/A
3860	 48, N/A
1050	 35, N/A
11550	 53, N/A
16160	 61, N/A
7720	 54, N/A
3130	 47, N/A
2880	 58, N/A
2370	 56, N/A
6730	 51, N/A
3410	 39, N/A
1110	 55, N/A
7260	 33, N/A
1390	 62, N/A
4220	 59, N/A
5250	 45, N/A
8840	 60, N/A
5770	 31, N/A
6550	 43, N/A
11180	 37, N/A
9900	 63, N/A
3250	 8, N/A
4040	 30, N/A
14280	 64, N/A
15050	 57, N/A
5590	 21, N/A
1260	 32, N/A
6720	  N/A, 45
3970	  N/A, 22
1220	  N/A, 38
2010	  N/A, 42
5600	  N/A, 29
1060	  N/A, 30
3180	  N/A